# 1，直接根据券原始数据，groupby 计算供应商维度的券分摊金额

In [2]:
from numpy import float64
import numpy as np
from datetime import datetime,date,time
import pandas as pd
from dq_work import rw_file
import imp
imp.reload(rw_file)#再次引入模块，默认只有一次，调试程序时有用
#选择第0个sheet，取到数据的第0行是字段名，从第1行开始是读取数据
rw_file1 = rw_file.rw_csv()


company='上海'
date_Range  ='20220701-0731'

#需要准备的导入文件清单
######################################
file_dir_0='e:/python-dq/work_data/tmp_data/in/'

file_1='01-用劵分摊-商品层级-原版-'+company+ '-all-'+date_Range+'.csv'

######################################
#导出的文件
file_out='供应商券分摊_out'+company+ '-all-'+date_Range+'.csv'


df_券金额原始数据 =  rw_file1.r_csv_utf8(file_dir_0+file_1,0,0)

#取得sku的券金额
g_大类_供应商=df_券金额原始数据.groupby(['大类编码','供应商业务码'])
df_大类_供应商_quan=pd.DataFrame(g_大类_供应商[['万家承担金额（去税）']].sum())
#取消索引
df_大类_供应商_quan.reset_index(inplace=True)
df_大类_供应商_quan.rename(columns={'万家承担金额（去税）':'券分摊金额'},inplace=True)


#导出供应商的分摊文件
rw_file1.w_csv(df_大类_供应商_quan,file_dir_0+file_out)

print('ok')



ok


# 2，根据供应商的销售占比，计算券分摊金额（待更新）

In [136]:
#导入数据
from numpy import float64
import pandas as pd
from dq_work import rw_file
from dq_work import work1
import imp
imp.reload(rw_file)#再次引入模块，默认只有一次，调试程序时有用
read_file = rw_file.rw_csv()

#配置导入源文件的目录与文件名
in_file_dir='e:/python-dq/work_data/毛利分析/202201/in/'

in_file1='03_业务报表-销售报表-销售-上海-sku-供应商.csv'
#in_file2='浙江采购收入数据_大类-202201.csv'
in_file3='12.上海用券汇总_out_202201.csv'


#配置导出结果的目录与文件名
out_file_dir='e:/python-dq/work_data/毛利分析/202201/out/'
out_file='2.上海供应商券分摊_out.csv'

df_单品销售原始数据 = read_file.r_csv_utf8(in_file_dir+in_file1,0,0)
df_券金额原始数据 =  read_file.r_csv_utf8(out_file_dir+in_file3,0,0)

print('over')



over


In [137]:
print(df_单品销售原始数据.columns)
print(df_券金额原始数据.columns)

Index(['省级公司名称', '渠道', '平台', '销售数量', '折扣净额', '销售金额', '销售净额(折前)', '销售净额',
       '销售成本', '补差', '生鲜损耗', '毛利额', '模块编码', '模块名称', '品类编码', '品类名称', '大类编码',
       '大类名称', '中类编码', '中类名称', '商品编码', '商品名称', '小类编码', '小类名称', '商品条码', '品牌名称',
       '子渠道', '供应商编码', '供应商名称'],
      dtype='object')
Index(['日期', '省级公司名称', '业态名称', '采购管理区域名称', '营运区域', '新旧店标识', '门店编码', '门店DC名称',
       '供应商编码', '供应商名称', '部门名称', '大类编码', '大类名称', '小类编码', '小类名称', '商品编码',
       '商品条码', '商品名称', '品牌名称', '渠道', '平台', '销项税率', '券类型编码', '券类型名称', '券号',
       '优惠类型', '小优惠类型', '用券总金额（去税）', '平台承担金额（去税）', '供应商承担金额（去税）',
       '万家承担金额（去税）'],
      dtype='object')


In [138]:
from dq_work import work1
import imp
imp.reload(work1)#再次引入模块，默认只有一次，调试程序时有用
analysis_1=work1.analysis_index()

#计算供应商的销售比例
g_sku_qudao_gys_xs =df_单品销售原始数据.groupby(['商品编码','大类编码','供应商编码'])
df_sku_qudao_gys_xs=pd.DataFrame(g_sku_qudao_gys_xs[['销售净额']].sum())
#取消索引
df_sku_qudao_gys_xs.reset_index(inplace=True)

bool2=df_sku_qudao_gys_xs['销售净额']>0
df_sku_qudao_gys_xs=df_sku_qudao_gys_xs[bool2]

                                 
g_sku_gys =df_sku_qudao_gys_xs.groupby(['商品编码','大类编码'])
df_quan_fentan=pd.DataFrame(g_sku_gys[['商品编码','大类编码','供应商编码','销售净额']] \
                               .apply(analysis_1.fentan_quan_gys))
#取消索引
#df_quan_fentan.reset_index(inplace=True)


#取得sku的券金额
g_sku_quan=df_券金额原始数据.groupby(['商品编码','大类编码'])
df_sku_quan=pd.DataFrame(g_sku_quan[['万家承担金额（去税）']].sum())
#取消索引
df_sku_quan.reset_index(inplace=True)



df_out1=pd.merge(df_sku_quan,df_quan_fentan,on=['商品编码','大类编码'],how='left')

#计算分摊金额
df_out1['券分摊金额']=df_out1['quan分摊比例']*df_out1['万家承担金额（去税）']

print('over')


over


In [139]:
bool3=df_out1['quan分摊比例'] == 0
bool4=df_out1['quan分摊比例'].isna()
bool5= bool3 | bool4
#取有效数据
df_out2=df_out1.loc[~bool5]

#===============
#取为空、为零的数据
df_out1_null=df_out1.loc[bool5]

#取得需要补充进来的券原始数据
df_券原始_null=pd.merge(df_券金额原始数据,df_out1_null,on=['商品编码','大类编码'])
#加工补充进来的数据
df_1=df_券原始_null.loc[:,['商品编码','大类编码','供应商编码_x','万家承担金额（去税）_x']]
g_tmp_1=df_1.groupby(['商品编码','大类编码','供应商编码_x'])
df_tmp1=pd.DataFrame(g_tmp_1[['万家承担金额（去税）_x']].sum())
df_tmp1.reset_index(inplace=True)
df_tmp1.rename(columns={'供应商编码_x':'供应商编码','万家承担金额（去税）_x':'券分摊金额'},inplace=True)
#=======================
#合并
df_out3=df_out2.append([df_tmp1])

In [140]:
#按照供应商编码进行汇聚

g_gys=df_out3.groupby(['供应商编码','大类编码'])

df_out=pd.DataFrame(g_gys[[ '券分摊金额']].sum())
#取消索引
df_out.reset_index(inplace=True)



In [141]:
df_out['券分摊金额'].sum()

29966262.966640234

## 导出数据

In [142]:
read_file.w_csv(df_out,out_file_dir+out_file)
print('over')

over
